# Configuring Azure ML Pipelines for Marine Debris

In [7]:
import azureml.core
from azureml.core import Workspace, Datastore
from azureml.core.authentication import InteractiveLoginAuthentication
auth = InteractiveLoginAuthentication(tenant_id = "d969963b-5316-4222-92ed-05c403954a09")
ws = Workspace.from_config(auth = auth)

WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


## 1st Pipeline Step: Access the data

In [15]:
datastores = ws.datastores
for name, datastore in datastores.items():
    print(f"'{name}'', {datastore.datastore_type}")

'workspaceblobstore'', AzureBlob
'workspacefilestore'', AzureFile
'coco'', AzureBlob
'coco_datastore'', AzureBlob
'debrisdatastore'', AzureBlob
'marinedestoragegtnaqtmj__hawaii_marine_debris'', AzureBlob
'marinedestoragegtnaqtmj__2015_dar_coastal_imagery'', AzureBlob


In [16]:
from azureml.core import Dataset, Datastore

datastore_name = "marinedestoragegtnaqtmj__2015_dar_coastal_imagery"
datastore = Datastore.get(ws, datastore_name)

In [18]:
mount = datastore.as_mount()

In [19]:
type(mount)

azureml.data.data_reference.DataReference

In [22]:
from azureml.core import Dataset, Run

annotations_ds = Dataset.get_by_name(ws, name='dar2015_kahoolawe_annotations')
df = annotations_ds.to_pandas_dataframe()
df

,filename,xmin,ymin,xmax,ymax,label
0,kahoolawe_107_9_7,121,0,418,244,None
1,kahoolawe_107_9_7,317,74,367,124,None
2,kahoolawe_107_9_7,277,0,512,286,None
3,kahoolawe_107_9_7,402,211,451,261,None
4,kahoolawe_107_9_8,9,214,58,264,None
5,kahoolawe_107_9_8,46,286,95,336,None
6,kahoolawe_107_9_8,298,232,348,282,None
7,kahoolawe_107_9_8,334,204,383,254,None
8,kahoolawe_107_9_14,299,9,348,59,None
9,kahoolawe_107_9_14,376,0,426,47,None


In [25]:
images_ds = Dataset.get_by_name(ws, name='2015dar_kahoolawe_pos_images')
images_ds

{
  "source": [
    "('marinedestoragegtnaqtmj__2015_dar_coastal_imagery', 'images/')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "name": "2015dar_kahoolawe_pos_images",
    "version": 1,
    "workspace": "Workspace.create(name='MarineDebrisWS', subscription_id='6d907faf-8cb3-413b-b1f1-688d0b36cb27', resource_group='MarineDebrisRG')"
  }
}